In [1]:
import cv2                                # state of the art computer vision algorithms library
import numpy as np                        # fundamental package for scientific computing
import matplotlib.pyplot as plt           # 2D plotting library producing publication quality figures
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
from realsense_depth import *
print("Environment Ready")

Environment Ready


In [2]:
net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt", "MobileNetSSD_deploy.caffemodel")
inScaleFactor = 0.007843
meanVal       = 127.53
classNames = ("background", "aeroplane", "bicycle", "bird", "boat",
              "bottle", "bus", "car", "cat", "chair",
              "cow", "diningtable", "dog", "horse",
              "motorbike", "person", "pottedplant",
              "sheep", "sofa", "train", "tvmonitor")

In [3]:
dc = DepthCamera()
#cv2.namedWindow("Color frame")
def midpoint (ptA, ptB):
     return int((ptA[0] + ptB[0]) * 0.5), int((ptA[1] + ptB[1]) * 0.5)
while True:
    ret, depth_frame, color_frame = dc.get_frame()
    height, width = color_frame.shape[:2]
    expected = 300
    aspect = width / height
    resized_image = cv2.resize(color_frame, (round(expected * aspect), expected))
    crop_start = round(expected * (aspect - 1) / 2)
    crop_img = resized_image[0:expected, crop_start:crop_start+expected]

    blob = cv2.dnn.blobFromImage(crop_img, inScaleFactor, (expected, expected), meanVal, False)
    net.setInput(blob, "data")
    detections = net.forward("detection_out")

    label = detections[0,0,0,1]
    conf  = detections[0,0,0,2]
    xmin  = detections[0,0,0,3]
    ymin  = detections[0,0,0,4]
    xmax  = detections[0,0,0,5]
    ymax  = detections[0,0,0,6]

    className = classNames[int(label)]
    
    #midpoint_val = midpoint((xmin, ymin), (xmax, ymax) )
    #distance = depth_frame[midpoint_val[1], midpoint_val[0]]
    # print("DIST in m", dist)
    #print("Detected a {0} {1:.3} meters away.".format(classNames[idx], distance))
    scale = height / expected
    xmin_depth = int((xmin * expected + crop_start) * scale)
    ymin_depth = int((ymin * expected) * scale)
    xmax_depth = int((xmax * expected + crop_start) * scale)
    ymax_depth = int((ymax * expected) * scale)
    #xmin_depth,ymin_depth,xmax_depth,ymax_depth
    depth = depth_frame[xmin_depth:xmax_depth,ymin_depth:ymax_depth].astype(float)
    depth= depth*(dc.scale)
    dist,_,_,_ = cv2.mean(depth)
    cv2.rectangle(crop_img, (int(xmin * expected), int(ymin * expected)),(int(xmax * expected), int(ymax * expected)), (255, 255, 255), 2)
    cv2.putText(crop_img, className+str(dist)+"m", (int(xmin * expected), int(ymin * expected) - 5),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255,255,255))
    cv2.imshow("crop_image",crop_img)
    print("Detected a {0} {1:.3} meters away.".format(className, dist))
    key = cv2.waitKey(1)
    if key == 27:
        cv2.destroyAllWindows()
        break

0.0010000000474974513
Detected a background 0.0 meters away.
Detected a aeroplane 2.4 meters away.
Detected a aeroplane 2.4 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a person 2.34 meters away.
Detected a sofa 2.42 meters away.
Detected a diningtable 2.41 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a person 2.65 meters away.
Detected a person 2.59 meters away.
Detected a background 0.0 meters away.
Detected a background 0.0 meters away.
Detected a backgr